In [ ]:
# imports
import pandas as pd
import pathlib
import json

In [ ]:
# read in raw data, downloaded direct messages from instagram, path to folder of folders where each contains the messages between the user and other users/groupchats
msgs = pathlib.Path("ig2024/messages")

In [ ]:
# declare empty dataframe
reel_data = pd.DataFrame()

In [ ]:
# format data into a dataframe
for msg in msgs.rglob("*/message_1.json"):
    with open(msg) as person:
        data = json.load(person)
        df = pd.DataFrame(data['messages'])
        if 'share' in df.columns:
            df = (df[(df['share'].apply(str).str.contains('instagram.com/reel/')) |
                  (df['share'].apply(str).str.contains('instagram.com/p/'))]
                  .assign(person = str(msg).split('/')[3])
                  [['person', 'sender_name', 'timestamp_ms']])
            reel_data = pd.concat([reel_data, df])
print(reel_data)

                                      person            sender_name  \
0                 drakeoaks_1286822379385550             Drake Oaks   
1                 drakeoaks_1286822379385550             Drake Oaks   
68    honeygreenmilkteaboba_1177352110328406  honeygreenmilkteaboba   
0                    lichat_1100965877971202               Licha T.   
2                    lichat_1100965877971202       ðkristinað¦   
...                                      ...                    ...   
9986                kewl_kev_454535175947162               kewl_kev   
9991                kewl_kev_454535175947162               kewl_kev   
9992                kewl_kev_454535175947162               kewl_kev   
9997                kewl_kev_454535175947162               kewl_kev   
9999                kewl_kev_454535175947162               kewl_kev   

       timestamp_ms  
0     1712611402148  
1     1709150466859  
68    1717399746079  
0     1708914298138  
2     1708890033836  
...            

In [ ]:
# convert timestamp to year-month
import datetime as dt
reel_data = reel_data.assign(time = (reel_data['timestamp_ms']/1000).apply(dt.datetime.fromtimestamp)).drop(columns=['timestamp_ms'])
reel_data = reel_data.reset_index(drop=True)
timed_reels = reel_data.assign(time = [str(t)[:7] for t in reel_data['time']])

In [ ]:
# cleaned dataframe, download to device
timed_reels.to_csv('reel_data.csv')
timed_reels

,person,sender_name,time
0,drakeoaks_1286822379385550,Drake Oaks,2024-04
1,drakeoaks_1286822379385550,Drake Oaks,2024-02
2,honeygreenmilkteaboba_1177352110328406,honeygreenmilkteaboba,2024-06
3,lichat_1100965877971202,Licha T.,2024-02
4,lichat_1100965877971202,ðkristinað¦,2024-02
...,...,...,...
5434,kewl_kev_454535175947162,kewl_kev,2024-05
5435,kewl_kev_454535175947162,kewl_kev,2024-05
5436,kewl_kev_454535175947162,kewl_kev,2024-05
5437,kewl_kev_454535175947162,kewl_kev,2024-05
